## How do I go about creating an optimization that LpMaximize profits and LpMinimize the variance?

https://stackoverflow.com/questions/60921824/python-pulp-how-do-i-create-lpmaximize-and-lpminimize-problem

20-11-2020

In [1]:
from pulp import *

In [72]:
# PROBLEM DATA:
costs =   [15, 25, 35, 40, 45, 55]
profits = [1.7, 2, 2.4, 3.2, 5.6, 6.2]
var =     [24, 12, 24, 32, 52, 62]
city = ["PORTO","COIMBRA","LISBOA","FARO","BRAGANÇA","GUARDA"] 

max_cost = 150
max_to_pick = 4

In [73]:
# DECLARE PROBLEM OBJECT:
prob = LpProblem("Mixed_Problem", LpMaximize)

In [74]:
# VARIABLES
n = len(costs)
N = range(n)
x = LpVariable.dicts('x', N, cat="Binary")

In [75]:
# OBJECTIVE
prob += lpSum([profits[i]*x[i] for i in N])

In [76]:
# CONSTRAINTS
prob += lpSum([x[i] for i in N]) == max_to_pick        #Limit number 
prob += lpSum([x[i]*costs[i] for i in N]) <= max_cost  #max cost

In [77]:
# NEW CONSTRAINT
for c in set(city):
    print(c)
    index_list = [i for i in N if city[i] == c] 
    prob += lpSum([x[i] for i in index_list]) <= 1

# SOLVE & PRINT RESULTS
prob.solve()

obj = value(prob.objective)


BRAGANÇA
PORTO
LISBOA
GUARDA
COIMBRA
FARO


In [78]:
x

{0: x_0, 1: x_1, 2: x_2, 3: x_3, 4: x_4, 5: x_5}

In [79]:
print('Status :', LpStatus[prob.status])
print('objective = ' + str(value(prob.objective)))
print('Profits ='+str(sum([x[i].varValue*profits[i] for i in N])))
print('Variance = ' + str(sum([x[i].varValue*var[i] for i in N])))
print('Cost = ' + str(sum([x[i].varValue*costs[i] for i in N])))

Status : Optimal
objective = 15.899999999999999
Profits =15.899999999999999
Variance = 162.0
Cost = 150.0


In [80]:
i = 0
route_city = 'Route City:'
for v in prob.variables():
    print (v.name, "=", v.varValue)
    if v.varValue > 0:  # Selected city
        route_city = route_city + '  > ' +str(city[i]) 
    i= i + 1
print(route_city)

x_0 = 1.0
x_1 = 0.0
x_2 = 1.0
x_3 = 0.0
x_4 = 1.0
x_5 = 1.0
Route City:  > PORTO  > LISBOA  > BRAGANÇA  > GUARDA


In [81]:
# MODIFY PROBLEM FOR 2ND PROBLEM

prob.sense = LpMinimize # change sense to LpMinimize
prob += lpSum([var[v]*x[v] for v in N]) # Reset the objective
prob += lpSum([profits[i]*x[i] for i in N]) == obj #Add constraint fixes profits 

In [82]:
# SOLVE 2ND PROBLEM
prob.solve()

print('Status :', LpStatus[prob.status])
print('objective = ' + str(value(prob.objective)))
print('Profits ='+str(sum([x[i].varValue*profits[i] for i in N])))
print('Variance = ' + str(sum([x[i].varValue*var[i] for i in N])))
print('Cost = ' + str(sum([x[i].varValue*costs[i] for i in N])))

Status : Optimal
objective = 162.0
Profits =15.899999999999999
Variance = 162.0
Cost = 150.0


In [83]:
i = 0
route_city = 'Route City:'

for v in prob.variables():
    print (v.name, "=", v.varValue)
    if v.varValue > 0:  # Selected city
        route_city = route_city + '  > ' +str(city[i]) 
    i= i + 1
print(route_city)

x_0 = 1.0
x_1 = 0.0
x_2 = 1.0
x_3 = 0.0
x_4 = 1.0
x_5 = 1.0
Route City:  > PORTO  > LISBOA  > BRAGANÇA  > GUARDA
